# Azure OpenAI - multi-function calling, using openai Python package v1.x

### 1. Testing connectivity to Vehicle APIs
You need to start companion Flask Web app, by executing "python -m flask run" command.

In [1]:
# Importing required packages
import requests

# Defining Vehicle API endpoint
VEHICLE_URL = "http://localhost:5000"

In [2]:
# Checking the vehicle status
vehicle_api = f"{VEHICLE_URL}/status"

vehicle_status = requests.get(vehicle_api)
print(vehicle_status.json())

{'airconditioner': 'OFF', 'lights': 'OFF', 'radio': 'OFF', 'windows': 'DOWN'}


In [3]:
# Helper function to manage the vehicle controls
def vehicle_control(feature, action, status):
    vehicle_control_api = f"{VEHICLE_URL}/{feature}"
    headers = {"Content-Type": "application/json"}
    data = {action: status}
    vehicle_control_status = requests.post(
        vehicle_control_api,
        headers = headers,
        json = data
    )
    function_response = vehicle_control_status.json()
    return function_response

In [4]:
# Switching on the vehicle's air conditioner
vehicle_control("airconditioner", "switch", "ON")

{'airconditioner': 'ON'}

### 2. Defining available functions

In [5]:
# Set of function tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "set_vehicle_feature_on_off",
            "description": "Set or switch features like air conditioner, lights and radio on or off",
            "parameters": {
                "type": "object",
                "properties": {
                    "feature": {
                        "type": "string",
                        "enum": ["airconditioner", "lights", "radio"]
                    },
                    "status": {
                        "type": "string",
                        "enum": ["on", "off"]
                    },
                },
                "required": ["feature", "status"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_vehicle_feature_up_down",
            "description": "Set or roll features like windows up or down",
            "parameters": {
                "type": "object",
                "properties": {
                    "feature": {
                        "type": "string",
                        "enum": ["windows"]
                    },
                    "status": {
                        "type": "string",
                        "enum": ["up", "down"]
                    },
                },
                "required": ["feature", "status"]
            }
        }
    }
]

In [6]:
# Declaring available functions
available_functions = {
    "set_vehicle_feature_on_off": vehicle_control,
    "set_vehicle_feature_up_down": vehicle_control
}

### 3. Configuring parallel functions calling

In [7]:
"""
Define Azure OpenAI endpoint parameters
    - Set AOAI API Version to env variable OPENAI_API_VERSION
    - Set AOAI API Endpoint to env variable OPENAI_API_BASE
    - Set AOAI API Key to env variable OPENAI_API_KEY
    - Set AOAI Deployment Name to env variable OPENAI_API_DEPLOY
"""

# Importing required packages
from openai import AzureOpenAI
import os
import json

In [8]:
# Initiating Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("OPENAI_API_BASE"),
    api_key = os.getenv("OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION")
)

In [9]:
# Helper function to run the conversation
def run_conversation(messages):
    # Step 1: Sending the conversation and available functions to the model
    response = client.chat.completions.create(
        model = os.getenv("OPENAI_API_DEPLOY"), # model = "Azure OpenAI deployment name".
        messages = messages,
        tools = tools,
        tool_choice = "auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message    
    tool_calls = response_message.tool_calls

    # Fix as per the issue # https://github.com/openai/openai-python/issues/703
    response_message = json.loads(response.choices[0].message.model_dump_json())
    if response_message["content"] is None:
        response_message["content"] = ""
    if response_message["function_call"] is None:
        del response_message["function_call"]

    # Checking the response JSON structure
    # print(response.model_dump_json(indent=2))

    # Step 2: Checking if the model wanted to call a function
    if tool_calls:
        # Step 3: Extending conversation with a function reply        
        messages.append(response_message)

        # Step 4: Sending each function's response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            if function_name == "set_vehicle_feature_on_off":
                action = "switch"
            elif function_name == "set_vehicle_feature_up_down":
                action = "roll"
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                feature = function_args.get("feature"),
                status = function_args.get("status"),
                action = action
            )
            if function_response:
                function_response = f"{list(function_response.keys())[0]} is {list(function_response.keys())[0]}"
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                }
            )
        # Step 5: Sending the updated conversation to the model
        second_response = client.chat.completions.create(
            model = os.getenv("OPENAI_API_DEPLOY"), # model = "Azure OpenAI deployment name".
            messages=messages
        )
        return second_response

### 4. Practical testing of parallel function calling
This part may fail, if you use outdated version of pydantic package.
Can be resolved with **pip install --upgrade pydantic**, as per this reported issue: https://github.com/openai/openai-python/issues/703. 

In [10]:
# Submitting a completion request to Azure OpenAI GPT model v1106
messages = [
    {"role": "system", "content": "You are a smart in-car assistant. Your listen to commands and control vehicle features like air conditioner, lights, radio and windows."},
    {"role": "user", "content": "Please, switch off the air conditioner and roll the windows down."}
]
final_response = run_conversation(messages)
print(final_response.choices[0].message.content)

The air conditioner is now switched off, and the windows have been rolled down. Enjoy the breeze!
